In [1]:
pip install -q yahoo_fin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 1.9 MB/s eta 0:00:00


In [2]:
mkdir -p data results log

In [4]:
pip install  git+https://github.com/ahsank/runml#egg=runml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ahsank/runml to /tmp/pip-install-a70b490f/runml_05c139266809478a915dce5e66cf07ca
  Running command git clone --filter=blob:none --quiet https://github.com/ahsank/runml /tmp/pip-install-a70b490f/runml_05c139266809478a915dce5e66cf07ca
  Resolved https://github.com/ahsank/runml to commit c5b9d03de0a9426cffb1d5df2e5af98f92c80e23
  Preparing metadata (setup.py) ... done
  Created wheel for runml: filename=runml-0.0.5-py3-none-any.whl size=9584 sha256=dfea6f33149f5a77536829287bdf65041be351d6c9c8556e7868856cba51215e
  Stored in directory: /tmp/pip-ephem-wheel-cache-fr0r_c50/wheels/f2/b8/c3/4a7eff09ec7c25b87e926a860e211080cead60a23333b62d25
Successfully built runml


In [5]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [6]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [7]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers = ['ABNB', 'ACLS', 'AFRM' ,'AI', 'AMBA',
           'BILL', 'BMBL', 'CFLT', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU',
           'ESTC', 'ETSY', 'EXPE', 'GFS', 'GLBE', 'GOGO',
           'INMD', 'JKS', 'LYFT', 'MAXR', 'MDB',  'MNDY', 'MPWR',  'MQ', 'MXL',
           'MTCH', 'NET', 'NVAX', 'NVCR',
           'OKTA', 'OLED', 'OSTK',
           'PANW', 'PAR', 'PAYO', 'PD', 'PLUG', 'PINS', 'PTON', 'RBLX',
           'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM', 'SWAV',
           'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'WOLF', 'VEEV', 'Z', 'ZM']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-1b', mod, True, loss=lossfn)[0]

In [13]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

Ticker    Name      Buy     Sell    Total  Accuracy       Error  Gain  \
45   RBLX  RROnly   256.70   490.73   747.43  0.975904   25.201561 -0.08   
56      U  RROnly   445.97   865.84  1311.81  0.962264   24.128892  0.18   
50   STEM  RROnly   169.06   216.92   385.98  0.942308    7.219322  0.25   
20   GLBE  RROnly   127.78   250.80   378.58  0.959459   18.532326 -0.10   
59   WOLF  RROnly   910.06   758.40  1668.46  0.952128   31.210253 -0.09   
38   PANW  RROnly   859.27   788.21  1647.48  0.946809   46.816005 -0.11   
39    PAR  RROnly   384.67   498.24   882.91  0.946809   11.951084 -0.11   
25   MAXR  RROnly   451.02   267.26   718.28  0.936170    4.999154 -0.14   
47   SNOW  RROnly  1074.65  1396.08  2470.73  0.953271  118.988493 -0.07   
31   MTCH  RROnly   879.69   853.25  1732.94  0.952128   41.361192 -0.07   
5    BILL  RROnly  1562.19  1745.82  3308.01  0.937931   34.069185 -0.10   
27   MNDY  RROnly   448.91  1597.16  2046.07  0.957143   83.950971  0.05   
12   DDOG  RROnly   988.24   752.45  1740.69  0.923567   30.836939 -0.13   
26    MDB  RROnly  3585.48  3837.49  7422.97  0.920213  105.579574 -0.12   
8    CRCT  RROnly    36.31   115.82   152.13  0.887500    6.797804  0.33   
52   TDOC  RROnly  1213.63  1380.55  2594.18  0.904255   26.143954  0.17   
16   ESTC  RROnly  1059.86   980.09  2039.95  0.941489   46.508368  0.06   
54    TTD  RROnly   856.78   547.46  1404.23  0.898936   16.200243 -0.18   
28   MPWR  RROnly  2939.71  2359.93  5299.63  0.909574  119.928022 -0.13   
24   LYFT  RROnly   343.88   460.90   804.78  0.933702   10.994008 -0.07   
34   NVCR  RROnly  1203.15  1202.08  2405.23  0.904255   45.450716  0.14   
1    ACLS  RROnly   470.65   260.32   730.97  0.888298   14.678095 -0.20   
57     UI  RROnly  1759.72  1675.77  3435.49  0.914894  110.471564 -0.10   
43   PINS  RROnly   420.87   325.79   746.66  0.910112   12.003695 -0.10   
44   PTON  RROnly   833.79   710.92  1544.71  0.903846    9.544089 -0.11   
40   PAYO  RROnly    28.03    41.42    69.45  0.901961    3.613447 -0.11   
21   GOGO  RROnly   159.18    82.78   241.96  0.920213    1.755227 -0.07   
7    CFLT  RROnly   185.08   295.70   480.78  0.941176   19.581415 -0.04   
42   PLUG  RROnly   425.75   235.24   660.99  0.835106    2.423863  0.37   
35   OKTA  RROnly  1545.77  1506.57  3052.34  0.920213   49.423392  0.06   
53   TOST  RROnly    68.40    93.78   162.18  0.964286   13.534651 -0.02   
22   INMD  RROnly   393.05   326.34   719.39  0.858896    8.172885  0.19   
33   NVAX  RROnly  1703.97  2171.92  3875.89  0.813830    8.968394  0.51   
3      AI  RROnly    74.23   255.81   330.04  0.852632   11.961433 -0.17   
17   ETSY  RROnly  1457.11  1176.45  2633.56  0.840426   35.997774  0.17   
62     ZM  RROnly  1460.29  2450.65  3910.94  0.842697   69.127349  0.14   
32    NET  RROnly   789.27   919.75  1709.02  0.822785   17.515816 -0.18   
15   DOCU  RROnly  1398.77  1195.20  2593.97  0.856383   43.994788 -0.10   
4    AMBA  RROnly  1148.16   849.33  1997.49  0.861702   40.150831 -0.09   
46   SNAP  RROnly   350.25   274.25   624.50  0.813830    9.111738  0.17   
13   DOCN  RROnly   348.34   394.16   742.50  0.901235   26.351701 -0.05   
14   DOCS  RROnly    82.85   387.55   470.40  0.867647   25.431847  0.08   
48   SPLK  RROnly  1083.83  1474.51  2558.34  0.909574   71.104531  0.04   
60   VEEV  RROnly  1668.53  1356.87  3025.40  0.925532  117.016617 -0.03   
0    ABNB  RROnly   375.80   608.65   984.45  0.852632   85.777228 -0.08   
41     PD  RROnly   226.29   358.91   585.20  0.905028   13.751196 -0.04   
11    DBX  RROnly   115.72   143.34   259.06  0.904255   16.060010  0.04   
6    BMBL  RROnly   163.77   223.08   386.85  0.895349   17.425887 -0.04   
2    AFRM  RROnly   397.73   770.01  1167.74  0.966667   11.548122 -0.01   
49   SPOT  RROnly  1633.54  1541.13  3174.67  0.893617   75.835607 -0.04   

      Last   Predicted     Alloc  
45   37.75   34.889999  9.457831  
56   36.29   42.689999  9.412998  
50    9.86   12.35000

In [ ]:
findata.EPOCHS=200
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX','GBTC', 
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MOAT', 'MTUM', 
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-12b', mod, True)
# !cp -r results /content/drive/MyDrive/colab

In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

Ticker    Name      Buy     Sell    Total  Accuracy       Error  Gain  \
0    ARKF  RROnly   205.79   230.90   436.69  0.956757   14.139005  0.37   
3    DAPP  RROnly    43.19   125.19   168.38  0.945946    3.035065  0.94   
35    XLE  RROnly   369.34   292.05   661.39  0.936170   21.525080 -0.15   
44   XNTK  RROnly   706.81   509.99  1216.80  0.968085   62.336498 -0.04   
12    IXC  RROnly   163.40   120.69   284.09  0.930851   11.581566 -0.16   
1    ARKK  RROnly   487.35   634.28  1121.63  0.920213   31.176111  0.23   
18    MGK  RROnly   899.67   611.44  1511.11  0.946809  108.947732 -0.07   
23    QQQ  RROnly  1278.43  1250.85  2529.28  0.946809  158.222218 -0.07   
32   XITK  RROnly   968.06   724.73  1692.80  0.941489   86.376755 -0.07   
43    XME  RROnly   280.19   221.25   501.44  0.930851   14.514160 -0.08   
24    SMH  RROnly  1445.98   804.34  2250.32  0.930851   89.256580 -0.08   
26    SPY  RROnly  1376.97  1147.06  2524.03  0.941489  216.855151 -0.05   
27   TDIV  RROnly   168.13   167.12   335.25  0.925532   30.037476 -0.07   
30    VTI  RROnly   667.97   721.58  1389.55  0.957447  108.327139 -0.03   
38    XLK  RROnly   510.81   593.26  1104.08  0.930851   61.675980 -0.06   
42    XLY  RROnly   756.51   591.23  1347.74  0.930851   86.988945 -0.06   
39   XLRE  RROnly   144.23   134.13   278.36  0.909574   23.620443 -0.09   
9    ICLN  RROnly   144.11   116.95   261.07  0.888298    8.482529 -0.13   
25   SMOG  RROnly   760.49   497.49  1257.98  0.941489   51.825745 -0.04   
17   JETS  RROnly    76.64   200.83   277.47  0.904255   12.356993  0.08   
15    IWM  RROnly   855.42   753.43  1608.86  0.914894   98.283690 -0.06   
5    DTEC  RROnly   181.56   160.12   341.68  0.946809   23.086692 -0.03   
28    VNQ  RROnly   298.93   292.02   590.94  0.888298   52.413088 -0.09   
20   MOAT  RROnly   213.05   208.51   421.57  0.925532   36.819560 -0.04   
11    IPO  RROnly   315.52   247.15   562.68  0.925532   22.261337 -0.04   
19    MGV  RROnly   337.77   224.41   562.19  0.920213   53.416807 -0.04   
34    XLC  RROnly   222.12   257.08   479.21  0.952128   39.850954 -0.02   
29     VT  RROnly   258.84   223.44   482.29  0.930851   52.128631 -0.03   
40    XLU  RROnly   227.50   144.33   371.83  0.898936   41.882675 -0.05   
14    IXP  RROnly   223.25   264.25   487.50  0.893617   46.152541 -0.05   
4     DIA  RROnly   914.51   914.94  1829.45  0.936170  179.259997  0.02   
6     EEM  RROnly   142.80   156.60   299.40  0.925532   29.283560  0.02   
10    IJR  RROnly   430.61   281.04   711.65  0.925532   48.080760 -0.02   
7     FPX  RROnly   469.29   460.39   929.68  0.957447   55.589184 -0.01   
45    XSW  RROnly   645.26   476.25  1121.51  0.920213   71.160083 -0.02   
31   WDIV  RROnly   179.67   173.49   353.16  0.914894   36.225067 -0.02   
21   MTUM  RROnly   606.78   385.60   992.38  0.914894   88.699517  0.02   
37    XLI  RROnly   347.19   238.43   585.62  0.904255   47.429138  0.02   
13    IXN  RROnly   236.32   163.62   399.94  0.936170   24.059191 -0.01   
41    XLV  RROnly   376.52   203.28   579.79  0.877660   72.554277 -0.02   
22   ONLN  RROnly   352.18   339.48   691.66  0.930851   27.121035  0.01   
8    GBTC  RROnly   366.24   305.58   671.83  0.867021    4.934133 -0.02   
33    XLB  RROnly   310.97   193.80   504.77  0.920213   37.065706 -0.01   
36    XLF  RROnly   123.82   101.39   225.22  0.904255   17.027573 -0.01   
16    IYZ  RROnly    74.43    89.02   163.45  0.877660   20.494017  0.01   
2    ARKW  RROnly   732.76   757.40  1490.16  0.925532   38.150193  0.00   

      Last   Predicted     Alloc  
0    14.27   19.559999  9.450694  
3     2.67    5.190000  9.401955  
35   87.47   74.529999  8.936170  
44   96.87   92.779999  8.882979  
12   38.99   32.700001  8.876330  
1    31.24   38.349998  8.855227  
18  172.07  159.800003  8.708207  
23  266.28  246.490005  8.708207  
32  100.23   93.300003  8.579027  
43   49.81   45.639999  8.444149  
24  202.94  186.710007  8.444149  
26  382.43

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# exclude = ['GTSWF', 'CBNT', 'ILAL', 'TNXP']
tickers3 = [ 'AIRI', 'APRN', 'AUGX', 'AUR', 'AHT', 'AMSC', 'ASTR', 'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM', 'CLOV', 'CREX', 'CZOO',
            'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FTCH',
            'GOVX', 'GTE', 'HIMS', 'HIVE', 
            'INLX', 'JOBY',  'KULR', 'LILM', 'LAZR', 'MMMB', 'MTTR', 'MULN',
            'NEPH', 'OMQS', 'ONDS', 'OPEN', 'OTLY',
            'RAD', 'REAL', 'RIDE', 'RIG', 'RVYL', 'SFIX', 'STEM',
            'SFT',  'TELL', 'VLDR', 'VRAR', 'WISH', 'YEXT']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-12a', mod, False)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3.sort_values('Alloc', ascending=False)

Ticker    Name     Buy     Sell    Total  Accuracy      Error  Gain   Last  \
16   DPSI  RROnly   19.09     6.05    25.14  0.954545   4.573494 -0.28   9.92   
49   YEXT  RROnly   91.17   121.35   212.52  0.904255   4.579372  0.28   6.21   
38   REAL  RROnly  118.88   175.74   294.63  0.884848   1.572779  0.50   1.17   
21   GOVX  RROnly   27.16    40.12    67.27  0.902913   0.711307  0.28   0.64   
40    RIG  RROnly   38.45    69.63   108.08  0.898936   0.828614 -0.30   4.90   
18   EXPR  RROnly   54.52    54.32   108.84  0.867021   0.779154  0.66   0.93   
5    AMSC  RROnly  119.53   153.58   273.11  0.909574   3.716450  0.20   3.63   
6    ASTR  RROnly   39.59    76.13   115.72  0.851485   0.786563  1.14   0.45   
7    ATOM  RROnly  195.12   233.40   428.52  0.904255   3.221546  0.19   6.24   
37    RAD  RROnly  146.52   178.23   324.75  0.872340   3.911866  0.40   3.45   
12   CLOV  RROnly   72.37    74.79   147.16  0.863248   1.246041  0.45   1.01   
44    SFT  RROnly   32.93    60.21    93.13  0.828402   0.348732  1.42   0.23   
1    APRN  RROnly  123.72   146.16   269.88  0.856383   1.299142  0.48   0.73   
19   FSLY  RROnly  585.15   558.77  1143.92  0.865497   9.162028  0.38   8.48   
43   STEM  RROnly  111.22   175.54   286.76  0.841584   7.372019  0.68   7.56   
9    BGSF  RROnly   85.79   101.01   186.80  0.898936   5.996851 -0.16  15.44   
31   MULN  RROnly   99.99   149.31   249.30  0.840426   0.529145  0.50   0.38   
34   ONDS  RROnly   23.13    56.49    79.62  0.838710   2.014711  0.40   1.53   
42   SFIX  RROnly  321.17   417.97   739.14  0.829787   4.048276  0.48   3.64   
23   HIMS  RROnly   58.22    81.65   139.87  0.832258   3.438408  0.41   6.13   
15   DMTK  RROnly  279.99   269.95   549.94  0.808511   2.651959  0.62   2.13   
48   WISH  RROnly   15.15    80.57    95.72  0.780220   0.737101  1.18   0.51   
32   NEPH  RROnly   47.28    63.82   111.10  0.845745   1.138903  0.26   1.11   
45   TELL  RROnly   41.34    65.03   106.37  0.861702   0.908878 -0.19   1.79   
47   VRAR  RROnly   39.99    33.32    73.31  0.765625   3.220771  1.33   2.85   
11   BSGM  RROnly   44.77    88.74   133.51  0.856383   0.494472  0.20   0.54   
26   JOBY  RROnly   23.09    52.57    75.66  0.843750   3.485211  0.19   3.37   
39   RIDE  RROnly   84.14   159.03   243.17  0.737143   1.266980  1.06   0.89   
8    BKKT  RROnly   97.59   115.67   213.26  0.747368   1.732964  0.81   1.33   
46   VLDR  RROnly   82.56   138.10   220.66  0.739362   1.118997  0.91   0.79   
29   MMMB  RROnly    5.52     7.04    12.56  0.951613   1.065570 -0.03   1.76   
30   MTTR  RROnly   70.36    88.40   158.76  0.811765   2.995207  0.23   2.72   
14   CZOO  RROnly    1.33    36.26    37.59  0.703297   0.391302  1.56   0.19   
0    AIRI  RROnly  101.55   123.11   224.66  0.824468   4.735902  0.19   4.38   
4     AHT  RROnly  613.87  1858.95  2472.82  0.686170  10.376424  2.69   4.32   
41   RVYL  RROnly   86.91    73.07   159.98  0.691489   0.447640  1.64   0.64   
3     AUR  RROnly   22.57    34.32    56.89  0.763889   1.455235  0.32   1.22   
28   LILM  RROnly    4.17    49.89    54.06  0.797872   1.346171  0.19   1.18   
27   KULR  RROnly   14.50    16.10    30.60  0.755319   0.927680  0.28   1.21   
36   OTLY  RROnly    5.40    45.28    50.68  0.814286   1.664143 -0.12   2.54   
2    AUGX  RROnly    6.93    26.91    33.84  0.831169   1.258642 -0.10   1.65   
17   EVGO  RROnly   31.18    70.52   101.70  0.705263   4.950508  0.31   3.89   
24   HIVE  RROnly  113.44   121.63   235.07  0.813830   0.703300 -0.11   1.82   
10   BLNK  RROnly  239.65   303.27   542.92  0.813830   2.266041 -0.11  11.26   
13   CREX  RROnly   17.74    22.16    39.90  0.755319   0.708748  0.16   0.60   
25   INLX  RROnly   60.31    53.36   113.67  0.771277   0.272827  0.12   4.08   
20   FTCH  RROnly  230.26   355.10   585.36  0.845745   5.316103  0.06   4.92   
33   OMQS  RROnly   46.82    58.13   104.95  0.648936   3.811361  0.11   5.00   
35   OPEN  RROnly  122.01   133.68

In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-1a', mod4, True)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

Ticker    Name       Buy     Sell     Total  Accuracy        Error  Gain  \
9     CAT  RROnly   1143.61   945.17   2088.78  0.936170    88.604945 -0.18   
6      BA  RROnly   1541.76  1949.73   3491.49  0.952128   100.177230 -0.10   
14     DE  RROnly   1994.12   960.20   2954.32  0.936170   110.816396 -0.10   
10   CDNS  RROnly    858.52   616.19   1474.71  0.920213    47.902398 -0.13   
1     ACN  RROnly   1172.50  1230.90   2403.40  0.914894   142.468332  0.14   
25   MSFT  RROnly   1318.61   626.05   1944.66  0.904255   102.162046  0.17   
24    MMC  RROnly    625.18   392.74   1017.92  0.925532    74.966745 -0.09   
31   QCOM  RROnly    832.26   720.28   1552.54  0.898936    47.068067 -0.15   
26    NKE  RROnly    680.95   532.15   1213.10  0.904255    63.410528 -0.12   
7    BKNG  RROnly  13606.40  9429.63  23036.03  0.941489  1182.328526 -0.05   
35    TXN  RROnly    715.06   532.43   1247.50  0.898936    88.155572 -0.11   
18    FDX  RROnly   1479.75   962.69   2442.44  0.898936    90.816982 -0.09   
2    ADBE  RROnly   2629.40  2202.96   4832.36  0.888298   246.103245 -0.10   
36    UNH  RROnly   1762.35   914.53   2676.89  0.882979   193.195546 -0.08   
3     AMD  RROnly    716.68   588.26   1304.94  0.888298    21.345078 -0.07   
11    CRM  RROnly   1235.17  1019.61   2254.78  0.882979   127.960168 -0.07   
27   NVDA  RROnly   1470.28  1400.49   2870.77  0.893617    36.933977  0.06   
22   ISRG  RROnly   1500.63  1654.38   3155.01  0.920213   127.235377 -0.04   
5    AVGO  RROnly   2304.42  1905.93   4210.35  0.867021   158.925924 -0.08   
34   TSLA  RROnly   2434.91  1663.89   4098.80  0.861702    18.478523 -0.08   
13   CSCO  RROnly    184.20   220.84    405.04  0.914894    30.793966 -0.04   
21    IBM  RROnly    530.28   607.26   1137.55  0.882979    80.470464 -0.06   
30   PYPL  RROnly   1457.51  1159.32   2616.83  0.909574    71.535260  0.04   
15    DHR  RROnly   1338.99   667.23   2006.22  0.872340   107.159913  0.06   
29   ORCL  RROnly    368.36   176.37    544.73  0.893617    39.101318 -0.04   
20   INTC  RROnly    235.68   296.21    531.88  0.893617    25.460882 -0.04   
8   BRK-B  RROnly   1210.49   526.90   1737.39  0.904255   164.828876 -0.03   
0    AAPL  RROnly    761.60   435.63   1197.23  0.904255    39.005273 -0.03   
37    UPS  RROnly    995.33   562.29   1557.62  0.888298    82.262534 -0.03   
4    AMZN  RROnly    770.09   899.08   1669.17  0.957447    81.496761 -0.01   
23   META  RROnly   1518.71  1209.92   2728.63  0.861702    94.634995 -0.03   
32   SBUX  RROnly    439.94   258.95    698.89  0.861702    54.700393 -0.03   
16    DIS  RROnly    827.25   724.49   1551.74  0.898936    86.452003 -0.02   
17  GOOGL  RROnly    408.87   436.85    845.72  0.888298    53.583795  0.02   
28    NOW  RROnly   3269.19  2523.70   5792.89  0.904255   193.678805 -0.01   
33    TMO  RROnly   2132.62  1709.33   3841.95  0.893617   241.735586 -0.01   
12   COST  RROnly   2080.11  1491.00   3571.11  0.898936   202.208835 -0.01   
19     HD  RROnly   1196.75  1018.08   2214.83  0.867021   146.912372 -0.00   

       Last    Predicted     Alloc  
9    248.86   204.119995  6.332320  
6    213.00   192.050003  6.322188  
14   426.47   382.399994  6.048632  
10   159.66   138.380005  5.959200  
1    269.21   307.000000  5.927052  
25   224.93   263.720001  5.895763  
24   170.96   154.860001  5.783519  
31   115.34    98.190002  5.747214  
26   124.53   110.000000  5.661094  
7   2179.25  2062.780029  5.554711  
35   175.16   155.630005  5.502211  
18   185.77   168.470001  5.298041  
2    332.75   297.880005  5.227964  
36   490.00   451.940002  4.844225  
3     63.96    59.639999  4.749240  
11   140.51   130.380005  4.635258  
27   148.59   157.839996  4.609929  
22   270.39   260.119995  4.578267  
5    588.43   539.859985  4.530775  
34   113.06   104.199997  4.426292  
13    48.32    46.180000  4.407295  
21   143.70   135.210007  4.356636  
30    76.48    79.430000  4.236322  
15   252.49   267.750000  4.10334

In [ ]:
findata.EPOCHS=200
tickers5 = ['ABB', 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'JMIA', 'KNNGF', 'MELI', 'MGA', 
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NXPI', 'OTGLY', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SHOP', 'SIEGY', 'STLA', 'STNE',
            'TEAM', 'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-12a', mod5, False)[0]


In [ ]:
addAlloc(df5, 0.14)
df5.sort_values('Alloc', ascending=False).head(30)

Ticker    Name      Buy     Sell    Total  Accuracy       Error  Gain  \
33   JMIA  RROnly   206.33   350.96   557.29  0.898305    3.209729  0.39   
3   AMSSY  RROnly   104.93   146.95   251.88  0.920213    2.878382  0.19   
26  FQVLF  RROnly   181.03   172.03   353.06  0.904255    3.956963 -0.21   
64  TRYIY  RROnly    63.02    58.95   121.97  0.936170    7.519320 -0.11   
6   ADYEY  RROnly   134.74   120.72   255.46  0.941176    6.717745 -0.08   
32    HMC  RROnly   116.91   123.37   240.28  0.952128   20.135420  0.06   
1   AEXAF  RROnly   250.82   515.57   766.39  0.829787    8.609585  0.37   
49  RYCEF  RROnly     8.04    36.44    44.48  0.803191    0.868805  0.43   
45  OTGLY  RROnly    63.52   136.97   200.49  0.863014    4.503367  0.20   
18  EIFZF  RROnly   207.37   128.88   336.25  0.872340    7.868488 -0.16   
7     BAK  RROnly   133.02   123.76   256.77  0.925532    3.892285  0.07   
42  NHYDY  RROnly    45.02    28.86    73.88  0.914894    1.811593 -0.08   
9   BESIY  RROnly   559.95   330.70   890.65  0.898936   19.110645 -0.10   
16  DPSGY  RROnly   228.14   208.33   436.47  0.856383   19.590597 -0.15   
39    LYB  RROnly   455.62   499.22   954.84  0.936170   30.448501 -0.05   
21   EQNR  RROnly   131.81   137.76   269.56  0.904255    8.161678 -0.08   
70  ZURVY  RROnly   158.21   164.47   322.67  0.877660   23.693202 -0.10   
44   NXPI  RROnly   973.21   724.43  1697.64  0.835106   66.198271 -0.15   
50  SAFRF  RROnly   817.93   783.12  1601.05  0.904255   58.970866 -0.07   
38   RACE  RROnly  1005.54   761.94  1767.48  0.877660  111.377814 -0.09   
62  THQQF  RROnly   103.57   172.61   276.18  0.701299    4.621043  0.30   
43    NOK  RROnly    28.75    26.09    54.84  0.882979    2.543786 -0.07   
51  SFTBY  RROnly   200.34   179.87   380.22  0.893617   13.580949  0.06   
14     CX  RROnly    32.10    41.50    73.59  0.893617    1.728884 -0.06   
8   BASFY  RROnly    81.06    94.46   175.52  0.941489    9.017488 -0.03   
69  YARIY  RROnly   109.49    86.32   195.81  0.888298    9.635186 -0.06   
46  POAHF  RROnly   473.65   451.63   925.28  0.888298   34.323947 -0.06   
47  PROSF  RROnly   360.94   450.97   811.91  0.896104   43.778139 -0.05   
22   ERIC  RROnly    39.58    62.31   101.90  0.877660    5.501437  0.06   
59   TEAM  RROnly  1510.35  1622.26  3132.61  0.803191  102.372080 -0.11   

      Last   Predicted     Alloc  
33    4.02    5.580000  5.894953  
3     4.54    5.410000  5.733083  
26   23.34   18.549999  5.547112  
64   11.24   10.020000  5.526389  
6    14.88   13.730000  5.252101  
32   23.58   24.889999  5.205167  
1     9.11   12.490000  5.006983  
49    1.34    1.910000  4.821694  
45    7.53    9.050000  4.794521  
18   38.98   32.790001  4.635258  
7     9.26    9.880000  4.483283  
42    7.85    7.220000  4.407295  
9    69.55   62.599998  4.399696  
16   41.75   35.520000  4.202128  
39   93.73   89.480003  4.133739  
21   31.42   28.959999  4.065350  
70   48.11   43.480000  3.822188  
44  171.82  145.789993  3.766464  
50  138.17  129.130005  3.723404  
38  235.92  214.619995  3.550321  
62    4.20    5.440000  3.017934  
43    5.03    4.650000  2.963526  
51   22.93   24.370001  2.836879  
14    4.98    4.670000  2.836879  
8    14.31   13.940000  2.824215  
69   22.88   21.600000  2.621581  
46   60.21   56.900002  2.621581  
47   82.02   78.269997  2.244898  
22    6.06    6.420000  2.190983  
59  148.00  132.119995  2.158746

In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'CUBS', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-12a', mod6, True)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

Ticker    Name     Buy    Sell   Total  Accuracy      Error  Gain   Last  \
15    ILF  RROnly   90.03  168.52  258.55  0.920213  12.920510 -0.07  24.85   
19   XCEM  RROnly  103.37   77.33  180.70  0.893617  16.235565 -0.10  27.30   
12    EWY  RROnly  262.14  326.87  589.01  0.904255  38.418788 -0.07  62.42   
17    THD  RROnly  262.49  246.99  509.48  0.904255  46.254503 -0.07  80.22   
16   KEMQ  RROnly  119.48  149.40  268.88  0.909574  11.889213  0.06  16.27   
10    EWP  RROnly  104.96   85.91  190.87  0.898936  16.829511 -0.06  26.55   
3     FLN  RROnly   79.73   85.48  165.21  0.909574   9.140864 -0.05  17.94   
8     EWC  RROnly  101.24  127.58  228.82  0.930851  16.958599 -0.03  34.81   
7     EWI  RROnly  127.86  109.74  237.61  0.904255  16.889404 -0.04  29.62   
4      FM  RROnly   91.49   77.11  168.60  0.920213  18.448870 -0.03  26.59   
6     EWA  RROnly   95.69   58.73  154.42  0.877660  11.609224 -0.05  24.01   
11    EWW  RROnly  201.94  132.99  334.92  0.877660  23.950610 -0.05  56.68   
14    IDX  RROnly   80.45   77.46  157.91  0.936170  10.097122 -0.02  17.58   
1     EWD  RROnly  151.13  137.78  288.91  0.920213  19.994990 -0.02  35.37   
18    TUR  RROnly  140.02  111.90  251.93  0.877660  17.020538 -0.03  33.55   
9     EWM  RROnly   79.36   58.76  138.12  0.893617  17.323915 -0.02  23.45   
13    EWZ  RROnly  191.43  172.20  363.62  0.914894  17.164384 -0.01  29.51   
5     ECH  RROnly  119.32  169.22  288.54  0.898936  16.043413  0.01  27.36   
2    CUBS  RROnly   15.43   28.68   44.12  0.910448  18.726309  0.00  20.70   
0    ARGT  RROnly  209.20  176.48  385.68  0.930851  14.535230 -0.00  39.28   

    Predicted     Alloc  
15  23.100000  8.062310  
19  24.480000  7.872340  
12  58.189999  7.674772  
17  74.860001  7.674772  
16  17.250000  7.588652  
10  25.049999  7.304965  
3   17.059999  7.287234  
8   33.639999  7.003546  
7   28.299999  6.648936  
4   25.889999  6.542553  
6   22.870001  6.329787  
11  53.790001  6.329787  
14  17.290001  6.170213  
1   34.560001  5.212766  
18  32.459999  4.698582  
9   23.000000  3.617021  
13  29.129999  0.638298  
5   27.719999  0.000000  
2   20.799999  0.000000  
0   39.150002  0.000000

In [ ]:
findata.EPOCHS=200
tickers7 = ['AAL', 'AMR', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BLK', 'BGFV',
            'CBT', 'CCJ', 'CF', 'CI', 'CIVI', 'CC', 'CCL',
            'CG', 'CMI', 'COKE', 'COP', 'CSL',
            'DD', 'EMN', 'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB', 
            'LUMN', 'M', 'MMM', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK', 
            'PARA', 'PCAR', 'PETS', 'PNR', 'PSTL', 'PXD', 'RCL', 'RH', 'REGN',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SWK',
            'T', 'TCN', 'TGT', 'TDG', 'TWI', 'UAL', 'UBA', 'UNH',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']
           
mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-12b', mod7, False)[0]

In [ ]:
addAlloc(df7, 0.10, 1)
df7.sort_values('Alloc', ascending=False).head(50)

Ticker    Name      Buy     Sell    Total  Accuracy       Error  Gain  \
10    CCJ  RROnly   157.18   152.32   309.50  0.941489    5.831012 -0.14   
13     CC  RROnly   234.85   248.08   482.93  0.936170    7.152214 -0.17   
36     KD  RROnly    21.38    49.70    71.08  0.940000    8.620640 -0.14   
66    UAL  RROnly   395.99   515.02   911.01  0.920213   20.994859  0.19   
14    CCL  RROnly   176.65   302.88   479.53  0.893617    7.083242  0.36   
30    HPE  RROnly    86.41    78.13   164.54  0.941489    7.308604 -0.07   
68    UNH  RROnly  2098.28  1105.15  3203.43  0.936170  192.895015 -0.07   
67    UBA  RROnly    86.11    90.52   176.63  0.930851    7.493832 -0.08   
3    ARCH  RROnly   771.43   391.79  1163.22  0.893617   21.545698 -0.18   
0     AMR  RROnly   727.77   319.52  1047.29  0.883721   13.961962 -0.20   
18    COP  RROnly   672.35   335.08  1007.43  0.904255   21.715207 -0.12   
35    JWN  RROnly   242.23   290.54   532.77  0.914894   12.097941 -0.09   
47   PARA  RROnly   285.04   331.05   616.09  0.867021   11.317227 -0.26   
37    KHC  RROnly   211.18   108.15   319.32  0.930851   18.335643 -0.06   
44    OMC  RROnly   304.15   308.69   612.83  0.920213   42.985966 -0.07   
54     RH  RROnly  3129.68  2914.40  6044.08  0.882979   89.850624  0.12   
24    FIS  RROnly   490.00   586.17  1076.17  0.898936   58.710418 -0.08   
52    PXD  RROnly  1240.43   821.12  2061.55  0.872340   50.882071 -0.12   
65    TWI  RROnly    99.94    61.27   161.22  0.882979    1.374039 -0.10   
22      F  RROnly   108.12    83.35   191.47  0.920213    4.124066 -0.05   
43    NTR  RROnly   423.94   260.56   684.50  0.856383   23.827461 -0.14   
32     IP  RROnly   215.04   179.31   394.35  0.941489   22.755609  0.03   
8    BGFV  RROnly   152.69   139.87   292.56  0.872340    1.129758  0.10   
9     CBT  RROnly   361.52   255.86   617.38  0.914894   20.637806 -0.05   
21    EMN  RROnly   475.01   577.21  1052.22  0.925532   36.861560  0.04   
40      M  RROnly   151.02   158.20   309.23  0.845745    4.880262 -0.14   
28    GWW  RROnly  2526.64  1276.12  3802.76  0.898936  207.589703 -0.06   
41    MMM  RROnly   647.50   825.86  1473.35  0.920213  107.991871  0.04   
45    OLN  RROnly   329.32   197.66   526.98  0.872340    9.491027 -0.08   
38    KMB  RROnly   459.15   409.27   868.42  0.904255   99.588503 -0.05   
49   PETS  RROnly   153.38   143.43   296.81  0.867021   13.367514  0.08   
11     CI  RROnly  1374.00   786.32  2160.31  0.882979  128.947983 -0.06   
23    FCX  RROnly   192.69   192.51   385.20  0.909574    5.859353 -0.04   
58   SPTN  RROnly   125.97   124.73   250.70  0.909574    8.192476 -0.04   
6     BAX  RROnly   232.40   310.15   542.54  0.824468   45.214640  0.12   
50    PNR  RROnly   260.01   222.24   482.24  0.877660   22.940710 -0.06   
12   CIVI  RROnly   379.02   367.59   746.61  0.904255    9.066917 -0.04   
55   REGN  RROnly  3581.77  1934.53  5516.30  0.872340  284.443897 -0.06   
20     DD  RROnly   419.43   393.61   813.04  0.941489   27.944497 -0.02   
42    NFG  RROnly   227.39   167.29   394.69  0.882979   30.482746  0.05   
53    RCL  RROnly   665.66   702.38  1368.03  0.898936   24.389737  0.04   
71    WDC  RROnly   314.67   468.07   782.74  0.840426   30.603580  0.08   
63    TGT  RROnly   977.14   863.88  1841.02  0.861702   67.502731 -0.06   
39   LUMN  RROnly    58.48    67.51   125.99  0.845745    5.372403  0.07   
16    CMI  RROnly  1142.17   602.62  1744.79  0.936170   99.027238  0.02   
34    IVZ  RROnly   116.59   117.84   234.43  0.909574    6.508920  0.03   
59    SPG  RROnly   687.32   642.35  1329.67  0.882979   39.665105 -0.04   
29   HSII  RROnly   161.09   185.31   346.40  0.930851   17.885102 -0.02   
48   PCAR  RROnly   381.64   208.98   590.62  0.856383   47.099372 -0.05   
57   SCCO  RROnly   383.42   228.54   611.96  0.856383   21.933553 -0.05   

      Last   Predicted     Alloc  
10   26.30   22.559999  8.996960  
13   33.72   27.990000  8.986233  
36   12.30   10.56000

In [ ]:
findata.EPOCHS=200
tickers8 = [ 'GLD', 'ISHG', 'JJU', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-1b', mod8, True)[0]

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

Ticker    Name     Buy    Sell   Total  Accuracy       Error  Gain    Last  \
4     SLV  RROnly  115.57   60.71  176.28  0.835106   11.644796  0.06   22.02   
2     JJU  RROnly  151.63  155.51  307.14  0.750000   33.191270 -0.12   54.41   
7     TLT  RROnly  273.93  350.33  624.26  0.851064   93.464877  0.02  106.20   
0     GLD  RROnly  378.29  293.12  671.41  0.824468  122.156742 -0.01  179.29   
1    ISHG  RROnly   72.76   75.16  147.91  0.781915   63.744817 -0.01   70.97   
3    PDBC  RROnly   38.45   26.13   64.57  0.813830    6.684698 -0.01   15.09   
5    SOYB  RROnly   67.88   27.48   95.36  0.792553   13.679151 -0.02   27.70   
6     TIP  RROnly  111.31   93.24  204.55  0.845745   97.030024  0.00  108.65   
8     UUP  RROnly   35.33   21.08   56.42  0.829787   24.031703  0.01   27.44   
9    VTIP  RROnly   28.79   12.96   41.75  0.861702   41.454808  0.00   47.04   
10   WEAT  RROnly   35.45   22.30   57.75  0.803191    5.071561 -0.02    7.53   

     Predicted     Alloc  
4    23.260000  5.602837  
2    47.750000  5.416667  
7   108.510002  1.063830  
0   177.320007  0.000000  
1    70.519997  0.000000  
3    14.890000  0.000000  
5    27.209999  0.000000  
6   109.190002  0.000000  
8    27.760000  0.000000  
9    47.070000  0.000000  
10    7.380000  0.000000

In [8]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-12b', mod9, False)[0]

loading model from results/cry-12b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
1/1 [==============================] - 0s 19ms/step


In [9]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

Ticker    Name        Buy       Sell      Total  Accuracy         Error  \
4    ETH-USD  RROnly   25756.11    7846.47   33602.58  0.702128    433.880926   
2    DOT-USD  RROnly     202.70      54.58     257.28  0.638554      5.881544   
0    ADA-USD  RROnly       8.32       4.67      12.99  0.632979      0.179073   
1    BTC-USD  RROnly  196798.40  152612.28  349410.68  0.664894  12067.429659   
5    LTC-USD  RROnly     730.30     356.65    1086.96  0.611702     59.844002   
7    XRP-USD  RROnly       8.41       4.24      12.65  0.595745      0.281890   
3   DOGE-USD  RROnly       0.80       1.21       2.01  0.531915      0.028316   
6  MATIC-USD  RROnly       7.83       9.69      17.52  0.675532      0.149077   

   Gain      Last     Predicted     Alloc  
4  0.38   1601.72   2217.659912  4.231323  
2  0.82      6.51     11.810000  4.120314  
0  0.71      0.38      0.660000  4.004345  
1  0.38  23161.35  31930.099609  3.867381  
5  0.42     88.26    125.160004  3.444782  
7  0.41      0.41      0.580000  3.269331  
3  0.64      0.09      0.140000  3.068009  
6  0.18      1.17      1.380000  3.022627

In [ ]:
findata.EPOCHS=200
save = ['CF']
tickers10 = ['AAPL', 'AN', 'AZN', 'BJ', 'BLDR',
             'COST', 'CPRX', 'CROX', 'DHR', 'DLTR', 'DY',
             'ELF', 'EQNR',  'HRMY', 'HUM',  'LNTH',
             'MCK',  'MPWR', 'NVDA', 'ON', 'PWR', 'QQQ',  'SWAV', 
             'TSLA', 'UNH', 'URI', 'VRRM', 'XLE']
mod10 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df10 = pipeline.runModelCombined(tickers10, 'trade-8b1', mod10, False)[0]

loading model from results/trade-8b1-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [ ]:
addAlloc(df10, 0.07, 1)
df10.sort_values('Alloc', ascending=False)

In [ ]:
findata.G_LOOKUP_STEP = 5
mod11 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df11 = pipeline.runModelCombined(tickers10, 'trade-8b1', mod11, False)[0]


loading model from results/trade-8b1-sh-1-sc-1-sbd-0-seq-50-step-5-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [ ]:
addAlloc(df11, 0.07, 1)
df11.sort_values('Alloc', ascending=False)

In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
com-1b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,215,302 bytes  received 35 bytes  6,430,674.00 bytes/sec
total size is 44,975,952  speedup is 13.99
/content/drive/MyDrive/colab/results/cry-10b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
